In [359]:
import math
from numpy import pi, deg2rad, rad2deg
from modsim import *
import pandas as pd
# install Pint if necessary

try:
    from pint import UnitRegistry
except ImportError:
    !pip install pint
    
# import units
from pint import UnitRegistry
units = UnitRegistry()

Drag Coefficient Calc Stuff

In [360]:
download('https://github.com/AllenDowney/ModSim/raw/main/data/' +
         'baseball_drag.csv')

In [361]:
from pandas import read_csv

baseball_drag = read_csv('baseball_drag.csv')

In [362]:
mph_to_mps = (1 * units.mph).to(units.m/units.s).magnitude
speed = baseball_drag['Velocity in mph'] * mph_to_mps

In [363]:
C_d_series = make_series(speed, baseball_drag['Drag coefficient'])

In [364]:
drag_interp = interpolate(C_d_series)

Constants

In [365]:
# baseball mass (kg)
BASEBALL_MASS = 0.145

# baseballRadius (cm)
BASEBALL_RADIUS = 3.683

# baseball cross-sectional area m^2
CROSS_SECTIONAL_AREA = 0.00426

# air density (kg/m^3)
P = 1.225

# g
G = 9.80665

CL = 0.22


Algos

In [366]:
# baseball cross-sectional area
def CSA(r):
    CROSS_SECTIONAL_AREA = 0.25 * math.pi * (2 * radius) ** 2 / 100 ** 2
    # CROSS_SECTIONAL_AREA_M2 = CROSS_SECTIONAL_AREA / 100 ** 2
    return CROSS_SECTIONAL_AREA_M2

# Converting miles per hour to meters per second.
def msec(mph):
    msec = mph * (1609/3600)
    return msec

# Converting meters per second to miles per hour.
def mphsec(msec):
    mphsec = msec * 2.237
    return mphsec

# Defining a function that takes a mass and returns the gravitational force.
def GRAV_FORCE(m):
    GRAV_FORCE = m * G
    GRAV_FORCE

# Defining a function that takes a value in RPM and returns the value in rad/s.
def rad_sec(rpm):
    rad_sec = rpm * 2 * math.pi / 60
    return rad_sec
radsec = rad_sec

# This function is taking the rotational speed of the baseball and the velocity of the baseball and returning the spin factor.
def S(spin):
    spin = radsec / v0
    return spin

# This function is taking the rotational speed of the baseball and the velocity of the baseball and returning the spin factor in meters/sec.
def vr_msec(rpm):
    rads = rad_sec(rpm)
    v = BASEBALL_RADIUS * rads / 100
    return v

# Taking the radius of the baseball and the rotational speed of the baseball and returning the tangential speed of the baseball.
def tangential_speed(radius, rpm):
    v = radius * rotational_speed(rpm)
    return v

# Converting the tangential velocity from meters per second to miles per hour.
def vr_mph(rpm):
    rads = RAD_SEC(rpm)
    v = BASEBALL_RADIUS * rads / 100
    v = mphsec(v)
    return v

# This function is taking the rotational speed of the baseball and the velocity of the baseball and returning the spin factor.
def spin_factor(rpm, velocity):
    spin_factor = rotational_speed(rpm) / msec(velocity)
    return f'The spin factor of {rpm}rpm and {velocity}mph is {round(spin_factor, 4)}'

- - -

We need the forces that are acting on the baseball:
1. Downward
2. Backward
3. Per spin, wpward or downward and at right angles to the path of the ball
   1. Upward if backspin
   2. Downward if topspin

In [435]:
throw = {
    "mph": float("80.0"),
    "theta": float("45.0"),
    "omega_rpm": float("3200.0"),
    "backspin": "0",
    "time": float("8"),
    "y0": float("1.75")
}

In [447]:
# Converting the spin rate from revolutions per minute to radians per second.
throw["omega_radsec"] = throw["omega_rpm"] * 2 * math.pi / 60

# Converting the speed from miles per hour to meters per second.
throw["vt_msec"] = throw["mph"] * 1609 / 3600

# Calculating the x-component of the initial velocity.
throw["vx"] = throw["vt_msec"] * math.cos(throw["theta"])

# Calculating the y-component of the initial velocity.
throw["vy"] = throw["vt_msec"] * math.sin(throw["theta"])

# Converting the spin rate from radians per second to miles per hour
throw["vr_mph"] = BASEBALL_RADIUS * throw["omega_radsec"] / 100 * 2.237

# Converting the spin rate from radians per second to meters per second
throw["vr_msec"] = BASEBALL_RADIUS * throw["omega_radsec"] / 100

# This is the spin factor in meters per second.
throw["spin_factor"] = throw["vr_msec"] / throw["vt_msec"]

# Using the interpolation function to find the drag coefficient for the given speed.
throw["coeff_drag"] = float(drag_interp(throw["vt_msec"]))

# Calculating the drag force.
throw["drag_force"] = (
    -0.5 * throw["coeff_drag"] * P * CROSS_SECTIONAL_AREA * throw["vt_msec"] ** 2
)

# Calculating CL from spin force
if throw["spin_factor"] < 0.1:
    throw["lift_coefficient"] = 1.6 * throw["spin_factor"]
else:
    throw["lift_coefficient"] = 0.6 * throw["spin_factor"] + 0.1


# This is the Magnus force.
throw["magnus"] = (
    0.5 * throw["lift_coefficient"] * P * CROSS_SECTIONAL_AREA * throw["vt_msec"] ** 2
)

# This is the x value at t time
throw["xt"] = (
    throw["vx"] + float(throw["vt_msec"]) * math.cos(throw["theta"]) * throw["time"]
)

# This is the y value at t time
throw["yt"] = (
    throw["vy"]
    + throw["vt_msec"] * math.sin(throw["theta"]) * throw["time"]
    - (0.5 * G * throw["time"] ** 2)
)

throw["fx"] = (
    -math.pi
    / 2
    * P
    * BASEBALL_RADIUS**2
    * throw["vt_msec"]
    * (throw["lift_coefficient"] * throw["vy"] + throw["drag_force"] * throw["vx"])
)

if throw["backspin"] == 1:
    throw["fy"] = (
        -math.pi
        / 2
        * P
        * BASEBALL_RADIUS**2
        * throw["vt_msec"]
        * (throw["lift_coefficient"] * throw["vx"] - throw["drag_force"] * throw["vy"])
    ) - BASEBALL_MASS * G
else:
    throw["fy"] = (
        -math.pi
        / 2
        * P
        * BASEBALL_RADIUS**2
        * throw["vt_msec"]
        * (-throw["lift_coefficient"] * throw["vx"] - throw["drag_force"] * throw["vy"])
    ) - BASEBALL_MASS * G

# Time of flight
# throw["time_of_flight"] = math.sqrt(-1 * 2 * throw["y0"] / G)

# Final Calc for X at T
# d_vx/dt = -kv(C_d * v_x + C_l * v_y)
# where k = P * pi * r^2/(2m)
throw["k"] = P * math.pi * (BASEBALL_RADIUS**2 / 2 * BASEBALL_MASS)
throw["x_at_t"] = (-1 * throw["k"] * throw["vt_msec"]) * (
    throw["coeff_drag"] * throw["vx"] + throw["lift_coefficient"] * throw["vy"]
)



# Final Calc for Y at T
# d_vy/dt = kv(C_l * v_x - C_d * V_y) - G
# where k = d * pi * r^2/(2m)

In [454]:
check = {
    "m": float("57.0"),  #g
    "g": float("9.8"),  #g
    "d": float("1.21"),  #g
    "dia": float("65"),  #mm
    "C_d": float("0.5"),  #
    "v_0": float("15"),  #m/s
    "y_0": float("30")  #m   
}

In [456]:
# dv/dt = -kC_dv^2-g
check["k"] = P * math.pi * ((check["dia"]/2)**2 / 2 * check["m"])
check["straight_fall_time"] = (-check["k"] * check["C_d"] * check["v_0"] ** 2)

In [457]:
check

{'m': 57.0,
 'g': 9.8,
 'd': 1.21,
 'dia': 65.0,
 'C_d': 0.5,
 'v_0': 15.0,
 'y_0': 30.0,
 'k': 115850.40152886669,
 'straight_fall_time': -13033170.171997502}

In [448]:
throw

{'mph': 80.0,
 'theta': 45.0,
 'omega_rpm': 3200.0,
 'backspin': '0',
 'time': 8.0,
 'y0': 1.75,
 'omega_radsec': 335.1032163829113,
 'vt_msec': 35.75555555555555,
 'vx': 18.78317955572727,
 'vy': 30.4245282438977,
 'vr_mph': 27.608721714638925,
 'vr_msec': 12.341851459382621,
 'spin_factor': 0.3451729743146165,
 'coeff_drag': 0.35869630078081605,
 'drag_force': -1.1965466924892565,
 'lift_coefficient': 0.3071037845887699,
 'magnus': 1.0244432878195955,
 'xt': 169.0486160015454,
 'yt': -39.99204580492068,
 'fx': 12255.077669746022,
 'fy': -28592.778104105255,
 'x_at_t': -2176.1182788591072,
 'k': 3.784664666735842}

In [419]:
throw

{'mph': 80.0,
 'theta': 45.0,
 'omega_rpm': 3200.0,
 'backspin': '1',
 'time': 8.0,
 'omega_radsec': 335.1032163829113,
 'vt_msec': 35.75555555555555,
 'vx': 18.78317955572727,
 'vy': 30.4245282438977,
 'vr_mph': 27.608721714638925,
 'vr_msec': 12.341851459382621,
 'spin_factor': 0.3451729743146165,
 'coeff_drag': 0.35869630078081605,
 'drag_force': -1.1965466924892565,
 'lift_coefficient': 0.3071037845887699,
 'magnus': 1.0244432878195955,
 'xt': 169.0486160015454,
 'yt': -39.99204580492068,
 'fx': 12255.077669746022,
 'fy': -28592.778104105255}

In [382]:
throw

{'mph': 90.0,
 'theta': 45.0,
 'omega_rpm': 2200.0,
 'backspin': '1',
 'time': 5.0,
 'omega_radsec': 230.3834612632515,
 'vt_msec': 40.225,
 'vx': 21.13107700019318,
 'vy': 34.22759427438491,
 'vr_mph': 18.98099617881426,
 'vr_msec': 8.485022878325552,
 'coeff_drag': 0.33219228689246594,
 'drag_force': -1.4024821487301837,
 'magnus': 0.928817690521875,
 'xt': 126.7864620011591,
 'yt': 82.78244064630945,
 'fx': 23209.373227134052}

In [410]:
throw

{'mph': 90.0,
 'theta': 45.0,
 'omega_rpm': 1200.0,
 'backspin': '1',
 'time': 8.0,
 'omega_radsec': 125.66370614359172,
 'vt_msec': 40.225,
 'vx': 21.13107700019318,
 'vy': 34.22759427438491,
 'vr_mph': 10.353270642989596,
 'vr_msec': 4.628194297268482,
 'spin_factor': 0.11505765810487215,
 'coeff_drag': 0.33219228689246594,
 'drag_force': -1.4024821487301837,
 'lift_coefficient': 0.16903459486292327,
 'magnus': 0.7136469182676418,
 'xt': 190.17969300173863,
 'yt': -5.76445153053578,
 'fx': 25040.874869378713,
 'fy': -54151.44875526274}

In [413]:
throw

{'mph': 90.0,
 'theta': 45.0,
 'omega_rpm': 2200.0,
 'backspin': '1',
 'time': 8.0,
 'omega_radsec': 230.3834612632515,
 'vt_msec': 40.225,
 'vx': 21.13107700019318,
 'vy': 34.22759427438491,
 'vr_mph': 18.98099617881426,
 'vr_msec': 8.485022878325552,
 'spin_factor': 0.21093903985893228,
 'coeff_drag': 0.33219228689246594,
 'drag_force': -1.4024821487301837,
 'lift_coefficient': 0.22656342391535936,
 'magnus': 0.9565278007172392,
 'xt': 190.17969300173863,
 'yt': -5.76445153053578,
 'fx': 22973.508886462238,
 'fy': -55427.77771435881}

In [416]:
throw

{'mph': 80.0,
 'theta': 45.0,
 'omega_rpm': 3200.0,
 'backspin': '1',
 'time': 8.0,
 'omega_radsec': 335.1032163829113,
 'vt_msec': 35.75555555555555,
 'vx': 18.78317955572727,
 'vy': 30.4245282438977,
 'vr_mph': 27.608721714638925,
 'vr_msec': 12.341851459382621,
 'spin_factor': 0.3451729743146165,
 'coeff_drag': 0.35869630078081605,
 'drag_force': -1.1965466924892565,
 'lift_coefficient': 0.3071037845887699,
 'magnus': 1.0244432878195955,
 'xt': 169.0486160015454,
 'yt': -39.99204580492068,
 'fx': 12255.077669746022,
 'fy': -39359.59204193811}

In [ ]:
# https://arimb.github.io/AMB_Design_Spreadsheet/Equations/AMB_Equations.pdf

In [343]:
math.pi / 2 * P * BASEBALL_RADIUS ** 2 * float(throw['vt_msec']) * ((CL * float(throw['vx'])) - (float(throw['coeff_drag']) * float(throw['vy']))) - (BASEBALL_MASS * G)

-7058.243203172633

In [ ]:
params = Params(
    x = 0,               # m
    y = release_point, 
    mph = 90,            # m
    angle = 45,          # degree
    speed = 44.34,       # m / s

    mass = 145e-3,       # kg 
    diameter = 73e-3,    # m 
    C_d = 0.33,          # dimensionless

    rho = 1.225,         # kg/m**3
    g = 9.806,           # m/s**2
    t_end = 10,          # s
)

In [1]:
from numpy import pi, deg2rad

def make_system(params):
    
    # convert angle to degrees
    theta = deg2rad(params.angle)
    
    # compute x and y components of velocity
    vx, vy = pol2cart(theta, params.speed)
    
    # make the initial state
    init = State(x=params.x, y=params.y, vx=vx, vy=vy)
    
    # compute the frontal area
    area = pi * (params.diameter/2)**2

    return System(params,
                  init = init,
                  area = area)

In [2]:
def slope_func(t, state, system):
    x, y, vx, vy = state
    mass, g = system.mass, system.g
    
    V = Vector(vx, vy)
    a_drag = drag_force(V, system) / mass
    a_grav = g * Vector(0, -1)
    
    A = a_grav + a_drag
    
    return V.x, V.y, A.x, A.y

In [3]:
def event_func(t, state, system):
    x, y, vx, vy = state
    return y

In [4]:
results, details = run_solve_ivp(system, slope_func,
                                 events=event_func)
details.message

NameError: name 'run_solve_ivp' is not defined

In [ ]:
def plot_trajectory(results):
    x = results.x
    y = results.y
    make_series(x, y).plot(label='trajectory')

    decorate(xlabel='x position (m)',
             ylabel='y position (m)')

plot_trajectory(results)

In [ ]:
def drag_force(V, system):
    rho, C_d, area = system.rho, system.C_d, system.area
    
    mag = rho * vector_mag(V)**2 * C_d * area / 2
    direction = -vector_hat(V)
    f_drag = mag * direction
    return f_drag

In [ ]:
vx, vy = system.init.vx, system.init.vy
V_test = Vector(vx, vy)
f_drag = drag_force(V_test, system)
show(f_drag)

In [ ]:
def slope_func(t, state, system):
    x, y, vx, vy = state
    mass, g = system.mass, system.g
    
    V = Vector(vx, vy)
    a_drag = drag_force(V, system) / mass
    a_grav = g * Vector(0, -1)
    
    A = a_grav + a_drag
    
    return V.x, V.y, A.x, A.y

In [ ]:
slope_func(0, system.init, system)

In [ ]:
def event_func(t, state, system):
    x, y, vx, vy = state
    return y

In [ ]:
event_func(0, system.init, system)

- - -

In [141]:
# Throw parameters
# x0 = feet; y0 = release point of hand in meters; v0 = release speed in meters/sec
mph = 90
x0 = 0
y0 = 1.75
v0 = msec(mph)
rpm = 2200
radsec = RAD_SEC(rpm)
angle = 45
time = 1
top = 0

In [142]:
# drag_force = - 0.5 * Cd * density * area * velocity ** 2  
# (in Newtons in direction opposite to current velocity)
drag_force = -0.5 * Cd * P * CSA(BASEBALL_RADIUS) * v0 ** 2
drag_force

-1.393687641180017

In [131]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

In [132]:
# drag coefficient
download('https://github.com/AllenDowney/ModSim/raw/main/data/' +
         'baseball_drag.csv')

baseball_drag = pd.read_csv('baseball_drag.csv')

mph_to_mps = (1 * units.mph).to(units.m/units.s).magnitude

speed = baseball_drag['Velocity in mph'] * mph_to_mps

C_d_series = make_series(speed, baseball_drag['Drag coefficient'])

drag_interp = interpolate(C_d_series)

array(0.31504741)

In [133]:
# Defining a function that takes a value of msec and returns the value of Cd at that point.
def Cd(msec):
    Cd = drag_interp(msec)
    return Cd

In [160]:
def throw(mph, rpm, angle, top):
    if top == 1:
        top = top * -1
    # Interpolating the drag coefficient for a baseball at v0 in m/s.
    Cd = drag_interp(msec(mph))
    # Calculating drag force
    drag_force = -0.5 * Cd * P * CSA(BASEBALL_RADIUS) * msec(mph) ** 2
    # Calculating the magnus force
    magnus = 0.5 * CL * P * CSA(BASEBALL_RADIUS) * msec(mph) ** 2
    # Calculating the ratio of the rotational speed of the ball to the translational speed of the ball.
    spin_factor = rotational_speed(rpm) / msec(mph)
    # Calculating the x component of the acceleration of the ball.
    ax = -1 * drag_force * math.cos(angle) - magnus * math.sin(angle)
    # Calculating the y component of the acceleration of the ball.
    ay = magnus * math.cos(angle) - drag_force * math.sin(angle) - G
    print(f'magnus: {magnus}')
    print(f'drag force: {drag_force}')
    print(f'Cd: {Cd}')
    print(f'ax: {ax}')
    print(f'ay: {ay}')
    print(f'spin factor: {spin_factor}')
    print(f'gravity: {G}')
    
    

In [161]:
throw(90, 2200, 45, 0)

magnus: 0.9291250941200111
drag force: -1.402946317385928
Cd: 0.33219228689246594
ax: -0.05359726766612638
ay: -8.124788191500567
spin factor: 0.21093903985893228
gravity: 9.80665


In [ ]:
# http://www.physics.usyd.edu.au/~cross/TRAJECTORIES/42.%20Ball%20Trajectories.pdf

In [ ]:
vx = -CSA(BASEBALL_RADIUS)*msec(mph)*((Cd * msec(mph)) + (CL * ))